In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
! pip install -q ..

^C
ERROR: Operation cancelled by user


In [5]:
import mouse_gait_analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from skimage import draw

from mouse_gait_analysis.io import *
from mouse_gait_analysis.utils import *
from mouse_gait_analysis.plots import *

In [7]:
data_folder = Path("/shared/thea/trial-analysis/")
! ls {data_folder}/*.h5

/shared/thea/trial-analysis/AccA19DLC_resnet50_A19allMar17shuffle1_1030000.h5
/shared/thea/trial-analysis/Day10A19DLC_resnet50_A19allMar17shuffle1_1030000.h5
/shared/thea/trial-analysis/Day1A19DLC_resnet50_A19allMar17shuffle1_1030000.h5
/shared/thea/trial-analysis/Day4A19DLC_resnet50_A19allMar17shuffle1_1030000.h5


In [24]:
results = []

raw_videos = list(data_folder.glob('*.mp4'))
raw_videos = list(filter(lambda x: 'labeled' not in x.parts[-1], raw_videos))
                  
for video in raw_videos:
    fname = video.parts[-1].split('.')[0]
    dlc_file = next(data_folder.glob(f'{fname}DLC*.h5'))

    video_analysis = VideoAnalysis(
        video,
        dlc_file)

    video_analysis.keypoints = filter_likelihood(video_analysis.keypoints, 0.8)
    video_analysis.keypoints = filter_distance_traveled(video_analysis.keypoints, 10, 1)
    
    # Registration
    video = Path(video_analysis.video).parts[-1]
    registration_points = pd.read_csv(data_folder / "registration_points.csv")
    registration_points['video'] = registration_points.video.apply(lambda x: Path(x).parts[-1])
    points = registration_points.groupby('video').get_group(video)[['y', 'x']].values
    w, h = 500, 500

    target = [[0,0], [w,0], [w,h], [0,h]]
    target = np.array(target)
    transformer = PerspectiveTransformer()
    transformer.register(points, target)
    keypoints = transformer.apply(video_analysis.keypoints)
    reader = VideoReader(video_analysis.video, transforms=[transformer])

    # Steps
    for bodypart in ['left_back_paw', 'right_back_paw']:
        bodypart_df = keypoints.xs(bodypart, axis=1, level='bodyparts').droplevel(0, axis=1)

        distances = []
        distance_threshold = 0.8
        d = 5
        for i in range(1, d):
            deltas = bodypart_df.diff(i).shift(-i)
            distance = np.log(np.sqrt(deltas.x**2 + deltas.y**2))
            distances.append(distance)
            
        distances = sum(distances) / len(distances)
        distance_forward = distances < distance_threshold

        distances = []
        for i in range(-d, 0):
            deltas = bodypart_df.diff(i)
            distance = np.log(np.sqrt(deltas.x**2 + deltas.y**2))
            distances.append(distance)
            
        distances = sum(distances) / len(distances)
        distance_backwards = distances < distance_threshold
        phase = distance_forward | distance_backwards

        # Correction
        # ----------
        step_df = pd.DataFrame(index=distance.index, columns=['state', 'episode'])
        step_df.state = phase.apply(lambda x: 'swing' if x else 'stance')
        episode = 0
        last = None
        segment_start = 0
        for idx in tqdm(step_df.index):
            current = step_df.loc[idx].state
            if current != last:
                step_df.loc[segment_start:idx, 'episode'] = episode
                episode += 1
                segment_start = idx
                last = current

        step_df.loc[segment_start:idx, 'episode'] = episode

        # Collapse small episodes
        min_episode_size = 3
        for episode in tqdm(sorted(step_df.episode.unique())):
            index = step_df[step_df.episode == episode].iloc[0].name
            if index-1 not in step_df.index:
                continue

            previous = step_df.loc[index-1]

            # Collapse small episodes
            if (step_df.episode == episode).sum() < min_episode_size:
                step_df.loc[step_df.episode == episode, 'episode'] = previous.episode
                step_df.loc[step_df.episode == episode, 'state'] = previous.state
                
            # Collapse side by side episodes of the same state
            if step_df.loc[index, 'state'] == previous.state:
                step_df.loc[step_df.episode == episode, 'episode'] = previous.episode

        # Renumber episodes
        episode_renumbering = {o:n for n,o in enumerate(sorted(step_df.episode.unique()))}
        step_df.episode = step_df.episode.apply(lambda x: episode_renumbering[x])

        # Episode output
        episode_df = pd.DataFrame(columns=['stride_length', 'duration', 'state', 'start', 'end'], index=sorted(step_df.episode.unique()))

        for episode, group in step_df.groupby('episode'):
            group = group.dropna()

            start_index = group.iloc[0].name
            end_index = group.iloc[-1].name

            deltas = bodypart_df.loc[[start_index,end_index]].diff().iloc[1]
            distance = np.sqrt(deltas.x**2 + deltas.y**2)

            episode_df.loc[episode, 'state'] = group.iloc[0].state
            episode_df.loc[episode, 'stride_length'] = distance
            episode_df.loc[episode, 'duration'] = end_index - start_index
            episode_df.loc[episode, 'start'] = start_index
            episode_df.loc[episode, 'end'] = end_index
            # TODO look at distance left/right along line

        episode_df = episode_df.dropna()

        # Remove outliers 
        episode_df['duration'] = episode_df['duration'].apply(lambda x: np.nan if x > 50 else x)
        episode_df['stride_length'] = episode_df['stride_length'].apply(lambda x: np.nan if x < 10 else x)
        episode_df['stride_length'] = episode_df['stride_length'].apply(lambda x: np.nan if x > 100 else x)

        # for column in ['stride_length', 'duration']:
        #     std = episode_df[column].std()
        #     mean = episode_df[column].mean()
        #     episode_df[column] = episode_df[column].apply(lambda x: np.nan if x > mean+3*std else x)

        episode_df = episode_df.dropna()

        swing = episode_df[episode_df.state == 'swing']
        print(f"{fname}, steps: {len(swing)} -- {bodypart}, "
              f"stride-length: {swing.stride_length.mean():.2f} +/- {swing.stride_length.std():.2f}, "
              f"duration: {swing.duration.mean():.2f} +/- {swing.duration.std():.2f}")

        result = dict(
            video=video,
            bodypart=bodypart,
            steps=len(swing),
            stride_length_median=swing.stride_length.median(),
            stride_length_mean=swing.stride_length.mean(),
            stride_length_var=swing.stride_length.var(),
            duration_median=swing.duration.median(),
            duration_mean=swing.duration.mean(),
            duration_std=swing.duration.std(),
        )
        results.append(result)

/usr/local/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
100%|██████████| 4839/4839 [00:21<00:00, 230.00it/s]
/usr/local/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


AccA19, steps: 23 -- left_back_paw, stride-length: 12.20 +/- 2.35, duration: 28.48 +/- 10.68


100%|██████████| 4895/4895 [00:24<00:00, 199.40it/s]


AccA19, steps: 37 -- right_back_paw, stride-length: 14.08 +/- 5.61, duration: 30.49 +/- 10.79


/usr/local/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
100%|██████████| 793/793 [00:03<00:00, 237.55it/s]
/usr/local/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


...

Day1A19, steps: 36 -- right_back_paw, stride-length: 13.75 +/- 3.83, duration: 28.50 +/- 10.15


/usr/local/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
100%|██████████| 3595/3595 [00:15<00:00, 233.36it/s]
/usr/local/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Day4A19, steps: 19 -- left_back_paw, stride-length: 12.41 +/- 2.04, duration: 27.47 +/- 11.98


100%|██████████| 3529/3529 [00:15<00:00, 234.49it/s]


Day4A19, steps: 31 -- right_back_paw, stride-length: 13.25 +/- 4.44, duration: 26.55 +/- 11.68
